In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.


In [ ]:
#导入训练数据&测试数据
train=pd.read_csv("../input/train.csv")
test=pd.read_csv("../input/test.csv")
test['casual']=0
test['registered']=0
test['count']=0
train = train[np.abs(train["count"]-train["count"].mean())<=(3*train["count"].std())] 

In [ ]:
train.shape,test.shape

In [ ]:
union_data=pd.concat([train,test],ignore_index=True)

In [ ]:

union_data['day']=pd.to_datetime(union_data.datetime).dt.day
union_data['year']=pd.to_datetime(union_data.datetime).dt.year
union_data['month']=pd.to_datetime(union_data.datetime).dt.month
union_data['weekday']=pd.to_datetime(union_data.datetime).dt.weekday
union_data['date']=pd.to_datetime(union_data.datetime).dt.date
union_data['hour']=pd.to_datetime(union_data.datetime).dt.hour
union_data['year_season']=union_data.apply(lambda x:'{}_{}'.format(str(x['year']),str(x['season'])),axis=1)
union_data['year_month']=union_data.apply(lambda x:'{}_{}'.format(str(x['year']),str(x['month'])),axis=1)
union_data['windspeed']=union_data[['year','month','hour','windspeed']].groupby(['year','month','hour']).transform(lambda x:x.replace(0,np.median(x)))


In [ ]:
train=union_data[:10739]

In [ ]:
#季节趋势
g=sns.factorplot(x='season',y='count',data=train,col='year',kind='bar',estimator=sum,ci=None,size=10,aspect=1)

In [ ]:
#季节趋势
g=sns.factorplot(x='month',y='count',data=train,col='year',kind='bar',estimator=sum,ci=None,size=10,aspect=1)

In [ ]:
#weekday 趋势
g=sns.factorplot(x='day',y='count',data=train,col='year',kind='bar',estimator=sum,ci=None,size=10,aspect=1)

In [ ]:
#weekday 趋势
g=sns.factorplot(x='weekday',y='count',data=train,col='year',kind='bar',estimator=sum,ci=None,size=10,aspect=1)

In [ ]:
#hour趋势
g=sns.factorplot(x='hour',y='count',data=train,col='year',kind='bar',estimator=sum,ci=None,size=10,aspect=1)

In [ ]:
g=sns.factorplot(x='weather',y='count',data=train,col='year',kind='bar',estimator=sum,ci=None,size=10,aspect=1)

In [ ]:
g=sns.factorplot(x='workingday',y='count',data=train,col='year',kind='bar',estimator=sum,ci=None,size=10,aspect=1)

In [ ]:
g=sns.factorplot(x='temp',y='count',data=train,col='year',kind='bar',estimator=sum,ci=None,size=10,aspect=1)

In [ ]:
train.columns

In [ ]:
date_season=train.groupby(['year','season'],as_index=False).agg({'count':sum})
date_season.columns=['year_time','season_time','count_s']

In [ ]:
date_season['year_season']=date_season.apply(lambda x:'{}_{}'.format(str(x['year_time']),str(x['season_time'])),axis=1)

In [ ]:
#季节趋势分析
plt.figure(figsize=(20,5))
sns.barplot(x='year_season',y='count_s',data=date_season)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
#X=train[['season', 'holiday', 'workingday', 'weather',
#      'atemp', 'humidity', 'windspeed','day', 'year', 'month', 'weekday','year_season']]
train_y=np.log1p(train[['count']])
undumm=union_data[['datetime','year_month','atemp', 'humidity', 'windspeed']]
get_dumm=union_data[[ 'workingday', 'weather','hour']]
#enc = OneHotEncoder()
#enc.fit(train[['season', 'holiday', 'workingday', 'weather','day', 'year', 'month', 'weekday']])
#enc.transform(train[['season', 'holiday', 'workingday', 'weather','day', 'year', 'month', 'weekday']]).toarray().shape
dumm=pd.get_dummies(get_dumm,columns=get_dumm.columns)
train_X=pd.concat([undumm[:10739],dumm[:10739]],axis=1)
test_X=pd.concat([undumm[10739:],dumm[10739:]],axis=1)


In [ ]:
regr = RandomForestRegressor(n_estimators=100)
regr.fit(train_X.loc[:,'year_month':], np.ravel(train_y))

In [ ]:
plt.figure(figsize=(100,5))
sns.barplot(x=train_X.loc[:,'year_month':].columns,y=regr.feature_importances_)

In [ ]:
union_data['count'][10739:]=np.exp(regr.predict(test_X.loc[:,'year_month':]))


In [ ]:
union_data[10739:].head(100)


In [ ]:
submission=pd.DataFrame({
        "datetime": union_data[10739:].datetime,
        "count": union_data[10739:]['count']
    })
submission.to_csv('bike_predictions_gbm_separate_without_fe.csv', index=False)